# Calculating EVPI and EVPPI from model outputs
#### Brandon Chan || Dec 2020

#### TODO:// Wrap into functions in analysis_helpers.py when finalized and debugged

### Import packages

In [3]:
import sys
sys.path.insert(0,"../src")

import pandas as pd
import numpy as np
from model_helpers import check_excel_file
from analysis_helpers import apply_new_costs, apply_new_utilities, get_state_mappings

### Load in existing model outputs

In [4]:
pop_base = np.load('../model_outputs/test_base_20-07-2020_population.npy')
cost_base = np.load('../model_outputs/test_base_20-07-2020_costs.npy')
util_base = np.load('../model_outputs/test_base_20-07-2020_utilities.npy')

pop_treat = np.load('../model_outputs/test_treat_20-07-2020_population.npy')
cost_treat = np.load('../model_outputs/test_treat_20-07-2020_costs.npy')
util_treat = np.load('../model_outputs/test_treat_20-07-2020_utilities.npy')

### Consense into per-iteration values of cost and utility

In [5]:
# Calculate effectivness of each iteration?
# consense into 1D array: [number of iterations]

#calculate sum at each time point (assign to bottom row)
# result array shape: [num_iterations, num_timesteps]
# each row is the sum of the costs/utilitys at time point t of an iteration
cost_sum_per_cycle_base = np.sum(cost_base, axis=1)
utility_sum_per_cycle_base = np.sum(util_base, axis=1)

cost_sum_per_cycle_treat = np.sum(cost_treat, axis=1)
utility_sum_per_cycle_treat = np.sum(util_treat, axis=1)

# Calculate the total cost of an iteration
cost_sum_per_iteration_base = np.sum(cost_sum_per_cycle_base, axis=1)
utility_sum_per_iteration_base = np.sum(utility_sum_per_cycle_base, axis=1)

cost_sum_per_iteration_treat = np.sum(cost_sum_per_cycle_treat, axis=1)
utility_sum_per_iteration_treat = np.sum(utility_sum_per_cycle_treat, axis=1)

## EVPI (with only two treatment options)

In [7]:
# Notes:
# net monetary benefit: threshold * change in effectivness - change in costs > 0 
# net health benefit: change in effectivness - (change in costs / threshold) > 0

# Columns needed:
# A = basecase (NB)
# B = treatment case (NB)
# Optimal choice of an iteration i = max_i(A,B)
# Maximum net benefit = max_i(A,B) (but the value itself)
# If B < A, oppertunity loss = A - B, else oppertunity loss = 0 
# ----------------------------------------------------------------
# Expectation = Avg(A) | Avg(B) | Avg(max next benefit) | Avg(oppertunity loss)

WTP_threshold = 20000

evpi_df = pd.DataFrame({'A':[],'B':[],'optimal_choice':[],'max_net_benefit':[],
                        'oppertunity_loss':[]})

population = 1
num_iterations = utility_sum_per_iteration_base.shape[0] # 1000
for i in range(0, num_iterations):
    
    A = (utility_sum_per_iteration_base[i] * WTP_threshold) - cost_sum_per_iteration_base[i]
    B = (utility_sum_per_iteration_treat[i] * WTP_threshold) - cost_sum_per_iteration_treat[i]
    
    max_net_benefit = max(A,B)
    
    if A >= B:
        opt_choice = 'A'
        oppertunity_loss = A - B
    else:
        opt_choice = 'B'
        oppertunity_loss = 0
       
    iteration_df = pd.DataFrame({'A':[A],'B':[B],'optimal_choice':[opt_choice],
                                 'max_net_benefit':[max_net_benefit],'oppertunity_loss':[oppertunity_loss]})
    evpi_df = evpi_df.append(iteration_df)

EVPI = evpi_df.oppertunity_loss.mean() * population

In [8]:
evpi_df.mean()

A                   58944.841127
B                   53591.082101
max_net_benefit     60969.445722
oppertunity_loss     7378.363621
dtype: float64

In [10]:
EVPI

7378.363620789668

## EVPPI

In [11]:
# This is more complicated and would require multiple model runs to conduct?
# i.e. fixing the value of certian parameters and only varying (sampling others)
# Theroterically possible by applying new costs/utilities to an existing population array output